# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [61]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [62]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [63]:
print(vocab_to_int)

{'.': 13, 'H': 36, 'm': 69, '&': 6, '1': 16, 'P': 44, 'e': 61, '$': 4, 'F': 34, '5': 20, 'a': 57, 'n': 70, "'": 7, 'h': 64, 'R': 46, 'w': 79, 'c': 59, 'E': 33, ';': 26, '?': 27, '9': 24, '\n': 0, ',': 11, 'q': 73, '*': 10, 'p': 72, 'O': 43, 'j': 66, 's': 75, '/': 14, 'd': 60, '@': 28, 'W': 51, 'N': 42, 'l': 68, 't': 76, '3': 18, 'U': 49, 'L': 40, '`': 56, 'B': 30, 'x': 80, 'k': 67, '7': 22, ' ': 1, 'i': 65, 'Q': 45, 'S': 47, ')': 9, 'b': 58, 'K': 39, '2': 17, '4': 19, 'g': 63, 'z': 82, 'A': 29, 'y': 81, 'Y': 53, 'I': 37, 'u': 77, 'M': 41, 'Z': 54, '!': 2, ':': 25, 'C': 31, '8': 23, '%': 5, 'v': 78, 'r': 74, 'D': 32, '(': 8, 'X': 52, '_': 55, '"': 3, 'J': 38, 'T': 48, 'o': 71, 'V': 50, '0': 15, 'f': 62, 'G': 35, '-': 12, '6': 21}


In [64]:
print(int_to_vocab)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: ',', 12: '-', 13: '.', 14: '/', 15: '0', 16: '1', 17: '2', 18: '3', 19: '4', 20: '5', 21: '6', 22: '7', 23: '8', 24: '9', 25: ':', 26: ';', 27: '?', 28: '@', 29: 'A', 30: 'B', 31: 'C', 32: 'D', 33: 'E', 34: 'F', 35: 'G', 36: 'H', 37: 'I', 38: 'J', 39: 'K', 40: 'L', 41: 'M', 42: 'N', 43: 'O', 44: 'P', 45: 'Q', 46: 'R', 47: 'S', 48: 'T', 49: 'U', 50: 'V', 51: 'W', 52: 'X', 53: 'Y', 54: 'Z', 55: '_', 56: '`', 57: 'a', 58: 'b', 59: 'c', 60: 'd', 61: 'e', 62: 'f', 63: 'g', 64: 'h', 65: 'i', 66: 'j', 67: 'k', 68: 'l', 69: 'm', 70: 'n', 71: 'o', 72: 'p', 73: 'q', 74: 'r', 75: 's', 76: 't', 77: 'u', 78: 'v', 79: 'w', 80: 'x', 81: 'y', 82: 'z'}


In [65]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [66]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [67]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [68]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs*n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:characters_per_batch*n_batches]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs,-1))
    print(arr)
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [69]:
#Visualize what is happpening

# In this code n_steps is the size of horizontal width of what moves
# And n_seqs is the no. of rows 
# y is the same size as x which is n_steps just one shifted right
n_seqs = 10
n_steps = 50
arr = encoded

characters_per_batch = n_seqs*n_steps
n_batches = len(arr)//characters_per_batch

arr = arr[:characters_per_batch*n_batches]
# Reshape into n_seqs rows
arr = arr.reshape((n_seqs,-1))
print(arr.shape)
print(arr)
for n in range(0, arr.shape[1], n_steps)[:1]:
    # The features
    x = arr[:, n:n+n_steps]
    # The targets, shifted by one
    y = np.zeros_like(x)
    y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
    print("x is: ", x)
    print("y is: ", y)

(10, 198500)
[[31 64 57 ..., 11  1 37]
 [ 1 57 69 ...,  1 40 61]
 [78 65 70 ..., 61 78 65]
 ..., 
 [26  1 58 ..., 81  1 65]
 [76  1 65 ..., 75 64 61]
 [ 1 75 57 ..., 65 71 70]]
x is:  [[31 64 57 72 76 61 74  1 16  0  0  0 36 57 72 72 81  1 62 57 69 65 68 65
  61 75  1 57 74 61  1 57 68 68  1 57 68 65 67 61 26  1 61 78 61 74 81  1
  77 70]
 [ 1 57 69  1 70 71 76  1 63 71 65 70 63  1 76 71  1 75 76 57 81 11  3  1
  57 70 75 79 61 74 61 60  1 29 70 70 57 11  1 75 69 65 68 65 70 63 11  1
  58 77]
 [78 65 70 13  0  0  3 53 61 75 11  1 65 76  7 75  1 75 61 76 76 68 61 60
  13  1 48 64 61  1 72 74 65 59 61  1 65 75  1 69 57 63 70 65 62 65 59 61
  70 76]
 [70  1 60 77 74 65 70 63  1 64 65 75  1 59 71 70 78 61 74 75 57 76 65 71
  70  1 79 65 76 64  1 64 65 75  0 58 74 71 76 64 61 74  1 79 57 75  1 76
  64 65]
 [ 1 65 76  1 65 75 11  1 75 65 74  2  3  1 75 57 65 60  1 76 64 61  1 71
  68 60  1 69 57 70 11  1 63 61 76 76 65 70 63  1 77 72 11  1 57 70 60  0
  59 74]
 [ 1 37 76  1 79 57 75  0 71 70

In [70]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

[[31 64 57 ..., 11  1 37]
 [ 1 57 69 ...,  1 40 61]
 [78 65 70 ..., 61 78 65]
 ..., 
 [26  1 58 ..., 81  1 65]
 [76  1 65 ..., 75 64 61]
 [ 1 75 57 ..., 65 71 70]]


In [71]:
print('x\n', x[:20, :20])
print('\ny\n', y[:20, :20])

x
 [[31 64 57 72 76 61 74  1 16  0  0  0 36 57 72 72 81  1 62 57]
 [ 1 57 69  1 70 71 76  1 63 71 65 70 63  1 76 71  1 75 76 57]
 [78 65 70 13  0  0  3 53 61 75 11  1 65 76  7 75  1 75 61 76]
 [70  1 60 77 74 65 70 63  1 64 65 75  1 59 71 70 78 61 74 75]
 [ 1 65 76  1 65 75 11  1 75 65 74  2  3  1 75 57 65 60  1 76]
 [ 1 37 76  1 79 57 75  0 71 70 68 81  1 79 64 61 70  1 76 64]
 [64 61 70  1 59 71 69 61  1 62 71 74  1 69 61 11  3  1 75 64]
 [26  1 58 77 76  1 70 71 79  1 75 64 61  1 79 71 77 68 60  1]
 [76  1 65 75 70  7 76 13  1 48 64 61 81  7 74 61  1 72 74 71]
 [ 1 75 57 65 60  1 76 71  1 64 61 74 75 61 68 62 11  1 57 70]]

y
 [[64 57 72 76 61 74  1 16  0  0  0 36 57 72 72 81  1 62 57 69]
 [57 69  1 70 71 76  1 63 71 65 70 63  1 76 71  1 75 76 57 81]
 [65 70 13  0  0  3 53 61 75 11  1 65 76  7 75  1 75 61 76 76]
 [ 1 60 77 74 65 70 63  1 64 65 75  1 59 71 70 78 61 74 75 57]
 [65 76  1 65 75 11  1 75 65 74  2  3  1 75 57 65 60  1 76 64]
 [37 76  1 79 57 75  0 71 70 68 81  1 79 64 61 

If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [72]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape = [batch_size, num_steps] , name ='inputs') #10*50 for our case
    targets =  tf.placeholder(tf.int32, shape = [batch_size, num_steps] , name ='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob =  tf.placeholder(tf.float32, name ='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [73]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    def build_cell(lstm_size, keep_prob):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)

        # Add dropout to the cell outputs
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    

    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [74]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis =1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x =tf.reshape(seq_output, ([-1, in_size]))
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name = 'predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [75]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =  tf.reshape(y_one_hot, (logits.get_shape()))
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(labels = y_reshaped, logits = logits)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [76]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [77]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [78]:
print(tf.__version__)

1.4.0


In [81]:
batch_size = 100         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [82]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
#                   'Training state: {:.4f}... '.format(new_state),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

[[31 64 57 ..., 75 11  1]
 [57 70 60 ..., 60 81 61]
 [78 65 76 ..., 77 68 60]
 ..., 
 [58 61 61 ..., 65 75  0]
 [57 68 79 ..., 76 57 70]
 [60  1 64 ..., 70 58 61]]
Epoch: 1/20...  Training Step: 1...  Training loss: 4.4192...  0.9449 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.3288...  0.9125 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.8004...  0.9031 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 5.6165...  0.8429 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 4.1094...  0.9063 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 4.0208...  0.9055 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.8245...  0.8885 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.6242...  0.8464 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.4959...  0.8483 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.4546...  0.8482 sec/batch
Epoch: 1/20...  Training Step: 11..

Epoch: 1/20...  Training Step: 101...  Training loss: 3.1128...  0.9252 sec/batch
Epoch: 1/20...  Training Step: 102...  Training loss: 3.1072...  0.9236 sec/batch
Epoch: 1/20...  Training Step: 103...  Training loss: 3.1104...  0.9246 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 3.0969...  0.9271 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 3.0965...  0.9276 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 3.0952...  0.9321 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 3.0712...  0.9453 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 3.0842...  0.9384 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 3.0885...  0.9366 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 3.0599...  0.9387 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 3.0736...  0.9374 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 3.0795...  0.9396 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 199...  Training loss: 2.5714...  0.9542 sec/batch
Epoch: 2/20...  Training Step: 200...  Training loss: 2.4928...  0.9558 sec/batch
Epoch: 2/20...  Training Step: 201...  Training loss: 2.5058...  0.9073 sec/batch
Epoch: 2/20...  Training Step: 202...  Training loss: 2.5103...  0.9040 sec/batch
Epoch: 2/20...  Training Step: 203...  Training loss: 2.4984...  0.9390 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.5035...  0.9519 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.5067...  0.9511 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.5066...  0.9544 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.5226...  0.9558 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.4886...  0.9554 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.4865...  0.9545 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.4891...  0.9531 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 299...  Training loss: 2.2826...  0.9565 sec/batch
Epoch: 2/20...  Training Step: 300...  Training loss: 2.2586...  0.9568 sec/batch
Epoch: 2/20...  Training Step: 301...  Training loss: 2.2271...  0.9578 sec/batch
Epoch: 2/20...  Training Step: 302...  Training loss: 2.2492...  0.9562 sec/batch
Epoch: 2/20...  Training Step: 303...  Training loss: 2.2399...  0.9547 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 2.2496...  0.9565 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 2.2432...  0.9550 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 2.2692...  0.9560 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 2.2504...  0.9570 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 2.2279...  0.9572 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 2.2436...  0.9576 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 2.2519...  0.9561 sec/batch
Epoch: 2/20...  

Epoch: 3/20...  Training Step: 397...  Training loss: 2.1690...  0.9714 sec/batch
Epoch: 3/20...  Training Step: 398...  Training loss: 2.0730...  0.9740 sec/batch
Epoch: 3/20...  Training Step: 399...  Training loss: 2.0833...  0.9758 sec/batch
Epoch: 3/20...  Training Step: 400...  Training loss: 2.0836...  0.9746 sec/batch
Epoch: 3/20...  Training Step: 401...  Training loss: 2.0967...  0.9141 sec/batch
Epoch: 3/20...  Training Step: 402...  Training loss: 2.0630...  0.9082 sec/batch
Epoch: 3/20...  Training Step: 403...  Training loss: 2.0897...  0.9599 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 2.0924...  0.9647 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 2.1170...  0.9661 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 2.0827...  0.9672 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 2.0596...  0.9671 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 2.0576...  0.9671 sec/batch
Epoch: 3/20...  

...

Epoch: 18/20...  Training Step: 3531...  Training loss: 1.2344...  0.8960 sec/batch
Epoch: 18/20...  Training Step: 3532...  Training loss: 1.2363...  0.8925 sec/batch
Epoch: 18/20...  Training Step: 3533...  Training loss: 1.2609...  0.8962 sec/batch
Epoch: 18/20...  Training Step: 3534...  Training loss: 1.2989...  0.8927 sec/batch
Epoch: 18/20...  Training Step: 3535...  Training loss: 1.2449...  0.8920 sec/batch
Epoch: 18/20...  Training Step: 3536...  Training loss: 1.2487...  0.8974 sec/batch
Epoch: 18/20...  Training Step: 3537...  Training loss: 1.2352...  0.8946 sec/batch
Epoch: 18/20...  Training Step: 3538...  Training loss: 1.2358...  0.8949 sec/batch
Epoch: 18/20...  Training Step: 3539...  Training loss: 1.2663...  0.8930 sec/batch
Epoch: 18/20...  Training Step: 3540...  Training loss: 1.2403...  0.8954 sec/batch
Epoch: 18/20...  Training Step: 3541...  Training loss: 1.2472...  0.8970 sec/batch
Epoch: 18/20...  Training Step: 3542...  Training loss: 1.2080...  0.8932 se

Epoch: 19/20...  Training Step: 3627...  Training loss: 1.2539...  1.0227 sec/batch
Epoch: 19/20...  Training Step: 3628...  Training loss: 1.2353...  0.8692 sec/batch
Epoch: 19/20...  Training Step: 3629...  Training loss: 1.2217...  0.8796 sec/batch
Epoch: 19/20...  Training Step: 3630...  Training loss: 1.2462...  0.8868 sec/batch
Epoch: 19/20...  Training Step: 3631...  Training loss: 1.2613...  0.8911 sec/batch
Epoch: 19/20...  Training Step: 3632...  Training loss: 1.2276...  0.8931 sec/batch
Epoch: 19/20...  Training Step: 3633...  Training loss: 1.2396...  0.8983 sec/batch
Epoch: 19/20...  Training Step: 3634...  Training loss: 1.2291...  0.8947 sec/batch
Epoch: 19/20...  Training Step: 3635...  Training loss: 1.2711...  0.8913 sec/batch
Epoch: 19/20...  Training Step: 3636...  Training loss: 1.2564...  0.8916 sec/batch
Epoch: 19/20...  Training Step: 3637...  Training loss: 1.2665...  0.8957 sec/batch
Epoch: 19/20...  Training Step: 3638...  Training loss: 1.2236...  0.8927 se

Epoch: 19/20...  Training Step: 3725...  Training loss: 1.2528...  0.8976 sec/batch
Epoch: 19/20...  Training Step: 3726...  Training loss: 1.2261...  0.8931 sec/batch
Epoch: 19/20...  Training Step: 3727...  Training loss: 1.2230...  0.8921 sec/batch
Epoch: 19/20...  Training Step: 3728...  Training loss: 1.2185...  0.8967 sec/batch
Epoch: 19/20...  Training Step: 3729...  Training loss: 1.2258...  0.8956 sec/batch
Epoch: 19/20...  Training Step: 3730...  Training loss: 1.2302...  0.8928 sec/batch
Epoch: 19/20...  Training Step: 3731...  Training loss: 1.2482...  0.8913 sec/batch
Epoch: 19/20...  Training Step: 3732...  Training loss: 1.2953...  0.8952 sec/batch
Epoch: 19/20...  Training Step: 3733...  Training loss: 1.2407...  0.8958 sec/batch
Epoch: 19/20...  Training Step: 3734...  Training loss: 1.2407...  0.8944 sec/batch
Epoch: 19/20...  Training Step: 3735...  Training loss: 1.2185...  0.8937 sec/batch
Epoch: 19/20...  Training Step: 3736...  Training loss: 1.2183...  0.8964 se

Epoch: 20/20...  Training Step: 3821...  Training loss: 1.1994...  0.9010 sec/batch
Epoch: 20/20...  Training Step: 3822...  Training loss: 1.2664...  0.9042 sec/batch
Epoch: 20/20...  Training Step: 3823...  Training loss: 1.2298...  0.9003 sec/batch
Epoch: 20/20...  Training Step: 3824...  Training loss: 1.2732...  0.9052 sec/batch
Epoch: 20/20...  Training Step: 3825...  Training loss: 1.2444...  0.9009 sec/batch
Epoch: 20/20...  Training Step: 3826...  Training loss: 1.2330...  0.9036 sec/batch
Epoch: 20/20...  Training Step: 3827...  Training loss: 1.2157...  0.9014 sec/batch
Epoch: 20/20...  Training Step: 3828...  Training loss: 1.2386...  0.9054 sec/batch
Epoch: 20/20...  Training Step: 3829...  Training loss: 1.2448...  0.9005 sec/batch
Epoch: 20/20...  Training Step: 3830...  Training loss: 1.2115...  0.9043 sec/batch
Epoch: 20/20...  Training Step: 3831...  Training loss: 1.2342...  0.9024 sec/batch
Epoch: 20/20...  Training Step: 3832...  Training loss: 1.2168...  0.9057 se

Epoch: 20/20...  Training Step: 3919...  Training loss: 1.2241...  0.8950 sec/batch
Epoch: 20/20...  Training Step: 3920...  Training loss: 1.2012...  0.8934 sec/batch
Epoch: 20/20...  Training Step: 3921...  Training loss: 1.1940...  0.8912 sec/batch
Epoch: 20/20...  Training Step: 3922...  Training loss: 1.2401...  0.8958 sec/batch
Epoch: 20/20...  Training Step: 3923...  Training loss: 1.2442...  0.8964 sec/batch
Epoch: 20/20...  Training Step: 3924...  Training loss: 1.2306...  0.8914 sec/batch
Epoch: 20/20...  Training Step: 3925...  Training loss: 1.2193...  0.8996 sec/batch
Epoch: 20/20...  Training Step: 3926...  Training loss: 1.2193...  0.8952 sec/batch
Epoch: 20/20...  Training Step: 3927...  Training loss: 1.2267...  0.8931 sec/batch
Epoch: 20/20...  Training Step: 3928...  Training loss: 1.2125...  0.8950 sec/batch
Epoch: 20/20...  Training Step: 3929...  Training loss: 1.2404...  0.8922 sec/batch
Epoch: 20/20...  Training Step: 3930...  Training loss: 1.2761...  0.8955 se

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [83]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i3960_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i3200_l512.ckpt"
all_model_checkpoint_pa

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [84]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [85]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [86]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i3960_l512.ckpt'

In [87]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i3960_l512.ckpt
Farmance when
they saw them to success at the caught the people to teal with her
thir sort of a corroce of his convictions.

"And he, what is the master," said the position of the porter, his
branches was beginning, turning walking about his soul, which were
assented into the country.

"I don't know that it, you're a letter without her, thinds one wants to
himself and their mother."

"I did not see how he could be touced of it."

"I had not at the simplicity, but he'll be simply been in her forgut
it without anyone; I'm going, but we would say that the mes in the
most of the party are no doubt. That won't then any obstacle towards to
her,'s such an obdering it were that the can talk in the country, and the
meanow--havo confidedly tell me. He wants to make his hand out of it, he
has never been told to the mistake, and he should be so fancied, and that
he must come to her that they were no musurce of, who has been atra

In [88]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Far lit othe asd whis touns oterere on ont whe an sat atereng he ton thir or tot wor the woth than sot ousd, wo ther, thit ande ting he hand sithes.""
"The ton the tered
sout tint war thes tamtit wot he satha se tant he the hereras hit hong ans withe tire theres,rend to the he wharerind. 
ho thon whe and whes an orins of an ha sis set int wath whe he the sathing
sard aldes ont the hem singitge son at antin tor atot ther she she whe the sos onderin the santhan se were than ton sere soth tot hind anghe he to hang as or te her hase and woth the sase tos the thas hesese her os toun the ansderad, tha se has ote tor at himen sist ang art whe aress as aseser tos tar tand at ho hos and onthar

the thind, bont ho than hore an addet alled, ha lor tin hess anle an her tore sot hon wand the ans oon he whan site wind wis hes
sith thin ser as tires ases oo hes ar arong ant th tathe wont ant the ter son shin sor this thes ande ho as

In [89]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Farry, and his
sacked anowher,
her sooded. She
had to she west and the shile the har said anding her word of a sone was thene what at she had to her that wert his sitt to becomed.

"Yes, wo love in with the mare," seimady She shonged him.

The collons in a cansing, that the carent the conse of a misint one as attine of the sincither and the cango on sheiles. How sall that in had seedent him
and and would strenghed the cersanted had the sand and hus on the hordine of in the
reant, hus the seepling the seres, and had buth him aly she had bees her.
There was her tone to the mathisting,
and to
back he him said. And said to heres the send that
it a censorseds of the himser fall the
selped ta sime.
"I and a that the moming to ander, as that him and the prosted tith
sheat that hen was sealing this to him, and a ponding, think homeng. There was to she would net intis the had and wish horded.

Ald homent of than torn tile on h

In [90]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Farly and well stranging or his featirn of
the met to her all, said to she he would have said to him his
been wene intand intintedly that some that in a corter of his head, but, he sunding at her hen, and had not been aster the prace of the side of the come of her
atried to see how the princess, and he was suppression when it was a camily and he say something as howing to
their attrood, and who had stardly as he would not tho sen a complouce and
his face and seening in this same with a can at happiness. He saw that the
stept of the drow while she was
all a side taking to the cause of this work, the cornes of the clurk a depirsable, and shall he could not the strow
of the pronert, that is with the care of the
stupe as his sither. Sotion of her subjutul of her, and she said in surdifiencle time and
here on hurrings. He went to bad his away, and
station of the sat trat what the
sore were interedat headt and ter in the h